In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout,SimpleRNN,Conv1D,MaxPooling1D,Flatten,Reshape
from tensorflow.keras.callbacks import EarlyStopping

# Load and process the dataset
data = pd.read_csv(r'C:\Users\shash\fetch_assessment\artifacts\data_ingestion\data_daily.csv')

# Adding additional features
data['# Date'] = pd.to_datetime(data['# Date'])
data['Day_of_Week'] = data['# Date'].dt.dayofweek
data['Month'] = data['# Date'].dt.month
data['Day'] = data['# Date'].dt.day
data['Year'] = data['# Date'].dt.year
data['Lag_1'] = data['Receipt_Count'].shift(1)
data['Lag_2'] = data['Receipt_Count'].shift(2)
data['Lag_3'] = data['Receipt_Count'].shift(3)
print(len(data))
# Dropping rows with NaN values after adding lag features
data.dropna(inplace=True)

# Scaling the features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data.drop(['# Date'], axis=1))

# Function to create dataset
def create_dataset(data, look_back):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:(i + look_back), :])
        y.append(data[i + look_back, 0])  # 0 index for 'Receipt_Count'
    return np.array(X), np.array(y)

# Creating the dataset with look back
look_back = 7
X, y = create_dataset(scaled_data, look_back)

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape, y_train.shape,X_test.shape,y_test.shape)

# Define the LSTM model
model = Sequential()
# CNN Layer
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(look_back, X.shape[2])))
# Removing MaxPooling1D layer to maintain the sequence length
# LSTM Layer
model.add(LSTM(50, activation='relu', return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50, activation='relu'))
model.add(Dropout(0.2))
# Output Layer
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping])
# Predicting on the test set
y_pred = model.predict(X_test)
print(y_pred.shape,y_test.shape)
# Calculating RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Test RMSE:", rmse)
# Reshape y_pred_scaled for inverse transformation
temp_shape = np.zeros((len(y_pred), scaled_data.shape[1]))
temp_shape[:, 0] = y_pred[:, 0]
y_pred = scaler.inverse_transform(temp_shape)[:, 0]

# Reshape y_test for inverse transformation
y_test_temp_shape = np.zeros((len(y_test), scaled_data.shape[1]))
y_test_temp_shape[:, 0] = y_test
y_test_rescaled = scaler.inverse_transform(y_test_temp_shape)[:, 0]

# Calculating RMSE on the rescaled data
rmse = np.sqrt(mean_squared_error(y_test_rescaled, y_pred))
print("Test RMSE on original scale:", rmse)


365
(284, 7, 8) (284,) (71, 7, 8) (71,)
Epoch 1/100
8/8 [==============================] - 3s 54ms/step - loss: 0.2052 - val_loss: 0.1238
Epoch 2/100
8/8 [==============================] - 0s 11ms/step - loss: 0.0613 - val_loss: 0.0202
Epoch 3/100
8/8 [==============================] - 0s 10ms/step - loss: 0.0287 - val_loss: 0.0116
Epoch 4/100
8/8 [==============================] - 0s 10ms/step - loss: 0.0159 - val_loss: 0.0125
Epoch 5/100
8/8 [==============================] - 0s 12ms/step - loss: 0.0137 - val_loss: 0.0058
Epoch 6/100
8/8 [==============================] - 0s 11ms/step - loss: 0.0112 - val_loss: 0.0042
Epoch 7/100
8/8 [==============================] - 0s 11ms/step - loss: 0.0108 - val_loss: 0.0036
Epoch 8/100
8/8 [==============================] - 0s 11ms/step - loss: 0.0108 - val_loss: 0.0038
Epoch 9/100
8/8 [==============================] - 0s 10ms/step - loss: 0.0104 - val_loss: 0.0036
Epoch 10/100
8/8 [==============================] - 0s 11ms/step - loss: 0.010

## LSTM CNN:0.07481992549716468
## LSTM: 0.08222140069384448
## RNN: 0.08422675959263284


In [1]:
# def predict_next_n_days(model, recent_data, n_days, scaler):
#     """
#     Predict the next 'n_days' days using the trained 'model' and 'recent_data'.

#     :param model: The trained forecasting model.
#     :param recent_data: A numpy array of the most recent data points (including features).
#     :param n_days: Number of days to predict into the future.
#     :param scaler: The MinMaxScaler used to scale the data.
#     :return: A list of predicted values for the next 'n_days' days.
#     """
#     future_predictions = []
#     current_batch = recent_data.copy()

#     for i in range(n_days):
#         # Predict the next time step
#         current_pred = model.predict(current_batch[np.newaxis, :])[0, 0]
        
#         # Append the prediction to the list
#         future_predictions.append(current_pred)
        
#         # Update the current batch to include the new prediction and update other features
#         current_batch = np.roll(current_batch, -1, axis=0)
#         # Update lagged Receipt_Count
#         current_batch[-1, 0] = current_pred
#         # Update other features if necessary (like date components)

#     # Rescale the predictions back to the original scale
#     temp_shape = np.zeros((len(future_predictions), scaled_data.shape[1]))
#     temp_shape[:, 0] = future_predictions
#     future_predictions_rescaled = scaler.inverse_transform(temp_shape)[:, 0]

#     return future_predictions_rescaled

# # Example usage
# recent_data = X_test[-1]  # Last row of the test data
# n_days = 10  # Number of days to predict
# predictions = predict_next_n_days(model, recent_data, n_days, scaler)


In [7]:
predictions

array([9443955.81066763, 9457532.1804018 , 9455146.96262801,
       9445791.14606917, 9444906.85578477, 9452504.65424836,
       9453441.9681499 , 9444955.44316304, 9460873.09078109,
       9460126.95765054])

In [2]:
import os

os.chdir("../")



In [5]:
from fetch_assessment.pipeline.predict import PredictionPipleline

In [6]:
d=PredictionPipleline()
d.predict(10)


[2023-11-13 22:46:01,437: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-13 22:46:01,437: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-13 22:46:01,441: INFO: common: created directory at: artifacts]
[2023-11-13 22:46:01,445: INFO: common: created directory at: artifacts/model_training]


1/1 [==============================] - 0s 19ms/step


array([9415045.26435351, 9419317.99614811, 9384337.61879206,
       9350605.51455843, 9331892.19144499, 9383879.62993956,
       9406431.98968422, 9408507.51573396, 9416416.4846679 ,
       9388182.35915899])